In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
pip install pyproj

     |████████████████████████████████| 10.9MB 2.8MB/s 


In [4]:
from pyproj import Geod

In [21]:
df=pd.read_excel('/content/drive/My Drive/Colab Notebooks/eg_table.xlsx')

In [6]:
df

,source,dst,hod,mean_travel_time,approx_travel_time,Source_Longitude,Source_Latitude,dst_Longitude,dst_Latitude
0,India Gate,Terminal 1 Indira Gandhi International Airport,14,18-30,24.0,77.229488,28.613082,77.1198,28.562864
1,India Gate,Terminal 1 Indira Gandhi International Airport,2,18-26,22.0,77.229488,28.613082,77.1198,28.562864
2,India Gate,Terminal 1 Indira Gandhi International Airport,4,18-24,21.0,77.229488,28.613082,77.1198,28.562864
3,India Gate,Terminal 1 Indira Gandhi International Airport,11,18-30,24.0,77.229488,28.613082,77.1198,28.562864
4,India Gate,Terminal 1 Indira Gandhi International Airport,20,20-30,25.0,77.229488,28.613082,77.1198,28.562864
5,India Gate,Terminal 1 Indira Gandhi International Airport,0,18-26,22.0,77.229488,28.613082,77.1198,28.562864
6,saraswati vihar,Mother teresa crescent,2,24-35,29.5,77.125276,28.696772,77.189588,28.617577
7,saraswati vihar,Mother teresa crescent,14,26-50,38.0,77.125276,28.696772,77.189588,28.617577
8,saraswati vihar,Mother teresa crescent,11,26-50,38.0,77.125276,28.696772,77.189588,28.617577
9,saraswati vihar,Mother teresa crescent,20,28-55,41.5,77.125276,28.696772,77.189588,28.617577


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   source              16 non-null     object 
 1   dst                 16 non-null     object 
 2   hod                 16 non-null     int64  
 3   mean_travel_time    16 non-null     object 
 4   approx_travel_time  16 non-null     float64
 5   Source_Longitude    16 non-null     float64
 6   Source_Latitude     16 non-null     float64
 7   dst_Longitude       16 non-null     object 
 8   dst_Latitude        16 non-null     float64
dtypes: float64(4), int64(1), object(4)
memory usage: 1.2+ KB


In [22]:
df['dst_Longitude']=df['dst_Longitude'].astype('float')

In [23]:
wgs84_geod = Geod(ellps='WGS84') #Distance will be measured on this ellipsoid - more accurate than a spherical method

#Get distance between pairs of lat-lon points
def Distance(lat1,lon1,lat2,lon2):
  az12,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) #Yes, this order is correct
  return dist

#Create test data
lat1 = np.random.uniform(-90,90,100)
lon1 = np.random.uniform(-180,180,100)
lat2 = np.random.uniform(-90,90,100)
lon2 = np.random.uniform(-180,180,100)

#Add/update a column to the data frame with the distances (in metres)
df['dist'] = Distance(df['Source_Latitude'].tolist(),df['Source_Longitude'].tolist(),df['dst_Latitude'].tolist(),df['dst_Longitude'].tolist())

In [24]:
df['distance in Km']=df['dist']*1000

In [26]:
df.drop(['dist'], axis=1, inplace=True)

In [27]:
labels_ordered=df.groupby(['hod'])['approx_travel_time'].mean().sort_values().index
labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
df['hod1']=df['hod'].map(labels_ordered)

In [28]:
if 0 in df['distance in Km'].unique():
    pass
else:
    df['distance in Km']=np.log(df['distance in Km'])

In [29]:
x=df.iloc[:,[9,10]]
y=df.iloc[:,4]

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [31]:
from sklearn.ensemble import RandomForestRegressor 
regressor = RandomForestRegressor()
regressor.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [32]:
y_pred=regressor.predict(x_test)
y_pred

array([24.12, 34.7 , 34.29, 35.04])

In [33]:
from sklearn.metrics import r2_score
score=r2_score(y_test,y_pred)

In [34]:
score

0.6220191096634091

In [35]:
y_pred2=regressor.predict(x)
y_pred2

array([24.68 , 24.12 , 21.63 , 23.26 , 25.13 , 22.02 , 34.7  , 34.86 ,
       34.29 , 35.04 , 34.09 , 49.25 , 51.1  , 51.675, 55.05 , 50.475])

In [36]:
df['Predicted_travel_time']=y_pred2

In [37]:
df.drop(['Source_Longitude'], axis=1, inplace=True)
df.drop(['Source_Latitude'], axis=1, inplace=True)
df.drop(['dst_Longitude'], axis=1, inplace=True)
df.drop(['dst_Latitude'], axis=1, inplace=True)
df.drop(['distance in Km'], axis=1, inplace=True)
df.drop(['hod1'], axis=1, inplace=True)

In [39]:
df

,source,dst,hod,mean_travel_time,approx_travel_time,Predicted_travel_time
0,India Gate,Terminal 1 Indira Gandhi International Airport,14,18-30,24.0,24.680
1,India Gate,Terminal 1 Indira Gandhi International Airport,2,18-26,22.0,24.120
2,India Gate,Terminal 1 Indira Gandhi International Airport,4,18-24,21.0,21.630
3,India Gate,Terminal 1 Indira Gandhi International Airport,11,18-30,24.0,23.260
4,India Gate,Terminal 1 Indira Gandhi International Airport,20,20-30,25.0,25.130
5,India Gate,Terminal 1 Indira Gandhi International Airport,0,18-26,22.0,22.020
6,saraswati vihar,Mother teresa crescent,2,24-35,29.5,34.700
7,saraswati vihar,Mother teresa crescent,14,26-50,38.0,34.860
8,saraswati vihar,Mother teresa crescent,11,26-50,38.0,34.290
9,saraswati vihar,Mother teresa crescent,20,28-55,41.5,35.040
